In [1]:
import numpy as np
import pandas as pd
from time import time

#from recsys.collaborative_deep_learing import DeepCollab

from keras.layers import Dense, Input, Concatenate
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, TensorBoard

/home/voldemort/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
train_ratings = pd.read_csv('data/train_ratings_set.csv')

In [4]:
train_ratings['rating'].value_counts()

4    1496995
5    1388429
3     959931
2     250846
1      86979
Name: rating, dtype: int64

In [6]:
train_ratings = train_ratings.pivot(index='user_id', columns='book_id')
train_ratings.head()

rating                                                        ...   \
book_id  1     2     3     4     5     6     7     8     9     10     ...    
user_id                                                               ...    
6          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    
7          NaN   NaN   NaN   NaN   3.0   NaN   NaN   2.0   NaN   NaN  ...    
8          NaN   NaN   NaN   3.0   3.0   NaN   NaN   NaN   NaN   1.0  ...    
9          4.0   4.0   4.0   NaN   NaN   NaN   NaN   NaN   1.0   5.0  ...    
10         NaN   NaN   NaN   5.0   NaN   NaN   NaN   NaN   NaN   NaN  ...    

                                                                     
book_id 9991  9992  9993  9994  9995  9996  9997  9998  9999  10000  
user_id                                                              
6         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
7         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
8         1.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
9         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
10        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 10000 columns]

In [7]:
train_ratings.fillna(0, inplace=True)

In [5]:
train_ratings_noisy = pd.read_csv('data/train_ratings_set.csv')

In [6]:
import random

random.seed(7)

# 5% noise
for _ in range(int(len(train_ratings_noisy) / 20)):
    i = random.randint(0, len(train_ratings_noisy))
    train_ratings_noisy.iloc[i]['rating'] = 0

In [7]:
train_ratings_noisy['rating'].value_counts()

4    1424262
5    1320247
3     913261
2     238649
0     204066
1      82695
Name: rating, dtype: int64

In [8]:
train_ratings_noisy = train_ratings_noisy.pivot(index='user_id', columns='book_id')
train_ratings_noisy.fillna(0, inplace=True)

In [17]:
train_ratings.to_csv('data/backups/train_ratings_pivot.csv', index=False)

In [9]:
train_ratings_noisy.to_csv('data/backups/train_ratings_noisy.csv', index=False)

In [14]:
train_ratings = pd.read_csv('data/backups/train_ratings_pivot.csv', header=0)

In [15]:
train_ratings_noisy = pd.read_csv('data/backups/train_ratings_noisy.csv', header=0)

In [16]:
train_ratings.shape

(53420, 10000)

In [21]:
train_ratings_noisy.shape

(53419, 10000)

In [22]:
train_ratings.head()

,rating,rating.1,rating.2,rating.3,rating.4,rating.5,rating.6,rating.7,rating.8,rating.9,...,rating.9990,rating.9991,rating.9992,rating.9993,rating.9994,rating.9995,rating.9996,rating.9997,rating.9998,rating.9999
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
train_ratings.drop(0, axis=0, inplace=True)
train_ratings_noisy.drop(0, axis=0, inplace=True)

In [23]:
train_ratings_noisy = np.asarray(train_ratings_noisy)
train_ratings = np.asarray(train_ratings)

# Generic Denoising Autoencoder
## 5 Star

In [28]:
model = DeepCollab(batch_size=128, hidden_layers=1, user_features=False)

In [29]:
model.fit(train_ratings_noisy, train_ratings)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 10000)             100010000 
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              10241024  
_________________________________________________________________
dense_6 (Dense)              (None, 10000)             10250000  
Total params: 120,501,024
Trainable params: 120,501,024
Non-trainable params: 0
_________________________________________________________________
Train on 42735 samples, validate on 10684 samples
Epoch 1/100
42735/42735 [==============================] - 31s 723us/step - loss: 3135.0102 - acc: 0.1445 - val_loss: 3057.5919 - val_acc: 0.1574
Epoch 2/100
42735/42735 [==============================] - 31s 7

In [30]:
model2 = DeepCollab(batch_size=128, hidden_layers=3, user_features=False)

In [31]:
model2.fit(train_ratings_noisy, train_ratings)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 10000)             100010000 
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              10241024  
_________________________________________________________________
dense_9 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_11 (Dense)             (None, 10000)             10250000  
Total params: 121,551,136
Trainable params: 121,551,136
Non-trainable params: 0
______________________________________________________________

In [32]:
model3 = DeepCollab(batch_size=256, hidden_layers=3, user_features=False)
model3.fit(train_ratings_noisy, train_ratings)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 10000)             100010000 
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              10241024  
_________________________________________________________________
dense_14 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_16 (Dense)             (None, 10000)             10250000  
Total params: 121,551,136
Trainable params: 121,551,136
Non-trainable params: 0
______________________________________________________________

In [33]:
model4 = DeepCollab(batch_size=64, hidden_layers=3, user_features=False)
model4.fit(train_ratings_noisy, train_ratings)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 10000)             100010000 
_________________________________________________________________
dense_18 (Dense)             (None, 1024)              10241024  
_________________________________________________________________
dense_19 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_20 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_21 (Dense)             (None, 10000)             10250000  
Total params: 121,551,136
Trainable params: 121,551,136
Non-trainable params: 0
______________________________________________________________

In [34]:
model5 = DeepCollab(batch_size=128, hidden_layers=5, user_features=False, nodes=2048)
model5.fit(train_ratings_noisy, train_ratings)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 10000)             100010000 
_________________________________________________________________
dense_23 (Dense)             (None, 2048)              20482048  
_________________________________________________________________
dense_24 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dense_25 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_26 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_27 (Dense)             (None, 2048)              2099200   
__________

In [37]:
model5.autoencoder.save('models/autoencoder_1stmethod.h5')

## 2nd Attempt at 'noise'

In [8]:
full_ratings = pd.read_csv('data/unprocessed/ratings.csv')

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
user_features = pd.read_csv('data/user_features_final.csv')
users = user_features['user_id'].tolist()

In [7]:
train_users, test_users = train_test_split(users, test_size=0.3)

In [8]:
data_users = pd.read_csv('data/train_ratings_set.csv')
train_data_users = data_users[data_users['user_id'].isin(train_users)]
test_data_users = data_users[data_users['user_id'].isin(test_users)]

In [9]:
train_data_users.to_csv('data/backups/autoencoder_train.csv', index=False)
test_data_users.to_csv('data/backups/autoencoder_test.csv', index=False)

In [10]:
train_data_users = train_data_users.pivot(index='user_id', columns='book_id')
train_data_users.fillna(0, inplace=True)
train_data_users = np.asarray(train_data_users)
train_data_users.shape

(37393, 10000)

In [11]:
denoised_train_data_users = full_ratings[full_ratings['user_id'].isin(train_users)]
denoised_train_data_users = denoised_train_data_users.pivot(index='user_id', columns='book_id')
denoised_train_data_users.fillna(0, inplace=True)
denoised_train_data_users = np.asarray(denoised_train_data_users)
denoised_train_data_users.shape

(37393, 10000)

In [14]:
model = DeepCollab(batch_size=128, hidden_layers=1, user_features=False)
model.fit(train_data_users, denoised_train_data_users)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10000)             100010000 
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              10241024  
_________________________________________________________________
dense_3 (Dense)              (None, 10000)             10250000  
Total params: 120,501,024
Trainable params: 120,501,024
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 29914 samples, validate on 7479 samples
Epoch 1/100
29914/29914 [==============================] - 12s 395us/step - loss: 3297.2810

In [15]:
model2 = DeepCollab(batch_size=128, hidden_layers=3, user_features=False)
model2.fit(train_data_users, denoised_train_data_users)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 10000)             100010000 
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              10241024  
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_8 (Dense)              (None, 10000)             10250000  
Total params: 121,551,136
Trainable params: 121,551,136
Non-trainable params: 0
______________________________________________________________

In [16]:
model3 = DeepCollab(batch_size=128, hidden_layers=5, user_features=False)
model3.fit(train_data_users, denoised_train_data_users)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 10000)             100010000 
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              10241024  
_________________________________________________________________
dense_11 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_13 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_14 (Dense)             (None, 1024)              525312    
__________

In [17]:
model3.autoencoder.save('models/autoencoder_full_noise_5layers.h5')

## Binary Training

In [18]:
train_data_users = data_users[data_users['user_id'].isin(train_users)]

train_data_users['rating'] = train_data_users.apply(lambda x: 1 if x['rating'] > 3 else 0, axis=1)
train_data_users = train_data_users.pivot(index='user_id', columns='book_id')
train_data_users.fillna(0, inplace=True)
train_data_users = np.asarray(train_data_users)
train_data_users.shape

/home/voldemort/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(37393, 10000)

In [19]:
denoised_train_data_users = full_ratings[full_ratings['user_id'].isin(train_users)]
denoised_train_data_users['rating'] = denoised_train_data_users.apply(lambda x: 1 if x['rating'] > 3 else 0, axis=1)
denoised_train_data_users = denoised_train_data_users.pivot(index='user_id', columns='book_id')
denoised_train_data_users.fillna(0, inplace=True)
denoised_train_data_users = np.asarray(denoised_train_data_users)
denoised_train_data_users.shape

/home/voldemort/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(37393, 10000)

In [20]:
model = DeepCollab(batch_size=128, hidden_layers=1, user_features=False)
model.fit(train_data_users, denoised_train_data_users)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 10000)             100010000 
_________________________________________________________________
dense_17 (Dense)             (None, 1024)              10241024  
_________________________________________________________________
dense_18 (Dense)             (None, 10000)             10250000  
Total params: 120,501,024
Trainable params: 120,501,024
Non-trainable params: 0
_________________________________________________________________
Train on 29914 samples, validate on 7479 samples
Epoch 1/100
29914/29914 [==============================] - 12s 390us/step - loss: 459.1833 - acc: 0.0682 - val_loss: 448.3762 - val_acc: 0.0558
Epoch 2/100
29914/29914 [==============================] - 12s 389u

In [21]:
model2 = DeepCollab(batch_size=128, hidden_layers=3, user_features=False)
model2.fit(train_data_users, denoised_train_data_users)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_19 (Dense)             (None, 10000)             100010000 
_________________________________________________________________
dense_20 (Dense)             (None, 1024)              10241024  
_________________________________________________________________
dense_21 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_22 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_23 (Dense)             (None, 10000)             10250000  
Total params: 121,551,136
Trainable params: 121,551,136
Non-trainable params: 0
______________________________________________________________

In [22]:
model3 = DeepCollab(batch_size=128, hidden_layers=5, user_features=False)
model3.fit(train_data_users, denoised_train_data_users)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 10000)             100010000 
_________________________________________________________________
dense_25 (Dense)             (None, 1024)              10241024  
_________________________________________________________________
dense_26 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_27 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_28 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_29 (Dense)             (None, 1024)              525312    
__________

In [24]:
model3.autoencoder.save('models/autoencoder_5layers_128batch.h5')

In [26]:
train_data_users.dump('data/backups/autoencoder_train_binary.csv')
denoised_train_data_users.dump('data/backups/autoencoder_train_denoised_binary.csv')

In [25]:
train_data_users = np.load('data/backups/autoencoder_train_binary.csv')
denoised_train_data_users = np.load('data/backups/autoencoder_train_denoised_binary.csv')

In [5]:
model4 = DeepCollab(batch_size=64, hidden_layers=5, user_features=False)
model4.fit(train_data_users, denoised_train_data_users)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10000)             100010000 
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              10241024  
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
dens

In [6]:
model4 = DeepCollab(batch_size=256, hidden_layers=5, user_features=False)
model4.fit(train_data_users, denoised_train_data_users)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 10000)             100010000 
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              10241024  
_________________________________________________________________
dense_10 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_12 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              525312    
__________

# User Inputs (CDAE)
## Binary

In [7]:
temp = pd.read_csv('data/backups/autoencoder_train.csv', header=0)
temp = temp.pivot(index='user_id', columns='book_id')
temp.head()

rating                                                        ...   \
book_id  1     2     3     4     5     6     7     8     9     10     ...    
user_id                                                               ...    
6          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    
7          NaN   NaN   NaN   NaN   3.0   NaN   NaN   2.0   NaN   NaN  ...    
8          NaN   NaN   NaN   3.0   3.0   NaN   NaN   NaN   NaN   1.0  ...    
9          4.0   4.0   4.0   NaN   NaN   NaN   NaN   NaN   1.0   5.0  ...    
10         NaN   NaN   NaN   5.0   NaN   NaN   NaN   NaN   NaN   NaN  ...    

                                                                     
book_id 9991  9992  9993  9994  9995  9996  9997  9998  9999  10000  
user_id                                                              
6         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
7         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
8         1.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
9         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
10        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 10000 columns]

In [10]:
train_users = temp.index.values

In [19]:
user_features = pd.read_csv('data/user_features_final.csv')
train_user_features = user_features[user_features['user_id'].isin(list(train_users))].copy()

In [20]:
train_user_features.shape

(37393, 27)

In [21]:
train_user_features.head()

,user_id,avg_rating,0,1,2,3,4,5,6,7,...,15,16,17,18,19,20,21,22,23,24
0,6.0,4.285714,0.195982,0.01082,0.008561,0.007327,0.03029,0.032636,0.028151,0.012982,...,0.001723,0.0,0.011177,0.020266,0.015214,0.020975,0.041102,0.010981,0.012653,0.06773
1,1006.0,3.784615,0.195982,0.01082,0.008561,0.007327,0.03029,0.032636,0.028151,0.012982,...,0.001723,0.0,0.011177,0.020266,0.015214,0.020975,0.041102,0.010981,0.012653,0.06773
2,2006.0,4.100000,0.195982,0.01082,0.008561,0.007327,0.03029,0.032636,0.028151,0.012982,...,0.001723,0.0,0.011177,0.020266,0.015214,0.020975,0.041102,0.010981,0.012653,0.06773
3,3006.0,3.819444,0.195982,0.01082,0.008561,0.007327,0.03029,0.032636,0.028151,0.012982,...,0.001723,0.0,0.011177,0.020266,0.015214,0.020975,0.041102,0.010981,0.012653,0.06773
10,10006.0,3.739726,0.195982,0.01082,0.008561,0.007327,0.03029,0.032636,0.028151,0.012982,...,0.001723,0.0,0.011177,0.020266,0.015214,0.020975,0.041102,0.010981,0.012653,0.06773


In [22]:
train_user_features['avg_rating'] = train_user_features.apply(lambda x: x['avg_rating']/5, axis=1)
train_user_features.drop('user_id', axis=1, inplace=True)

In [23]:
train_user_features = np.asarray(train_user_features)

In [38]:
train_data_users = train_data_users.pivot(index='user_id', columns='book_id')
train_data_users.fillna(0, inplace=True)
train_data_users = np.asarray(train_data_users)
train_data_users.shape

(37393, 10000)

In [16]:
denoised_train_data_users = full_ratings[full_ratings['user_id'].isin(train_users)]
denoised_train_data_users = denoised_train_data_users.pivot(index='user_id', columns='book_id')
denoised_train_data_users.fillna(0, inplace=True)
denoised_train_data_users = np.asarray(denoised_train_data_users)
denoised_train_data_users.shape

(37393, 10000)

In [35]:
model = DeepCollab(batch_size=128, hidden_layers=5, user_features=True)
model.fit(train_data_users, denoised_train_data_users, train_user_features)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 10000)        0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 26)           0                                            
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 10000)        100010000   input_10[0][0]                   
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 26)           702         input_11[0][0]                   
__________________________________________________________________________________________________
concatenat

In [36]:
model1 = DeepCollab(batch_size=128, hidden_layers=1, user_features=True)
model1.fit(train_data_users, denoised_train_data_users, train_user_features)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 10000)        0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 26)           0                                            
__________________________________________________________________________________________________
dense_30 (Dense)                (None, 10000)        100010000   input_12[0][0]                   
__________________________________________________________________________________________________
dense_31 (Dense)                (None, 26)           702         input_13[0][0]                   
__________________________________________________________________________________________________
concatenat

In [37]:
model2 = DeepCollab(batch_size=128, hidden_layers=3, user_features=True)
model2.fit(train_data_users, denoised_train_data_users, train_user_features)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 10000)        0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 26)           0                                            
__________________________________________________________________________________________________
dense_34 (Dense)                (None, 10000)        100010000   input_14[0][0]                   
__________________________________________________________________________________________________
dense_35 (Dense)                (None, 26)           702         input_15[0][0]                   
__________________________________________________________________________________________________
concatenat

In [39]:
model.autoencoder.save('models/autoencoder_userfeatures_5layers.h5')

In [40]:
train_user_features.dump('data/backups/train_user_features_autoencoder.csv')

## 5 Star

In [ ]:
train_data_users = pd.read_csv('data/backups/autoencoder_train.csv', header=0)
train_data_users['rating'] = train_data_users.apply(lambda x: train_data_users['rating']/5, axis=1)
train_data_users.head()

In [5]:
train_users = list(set(train_data_users['user_id'].tolist()))

In [6]:
train_user_features = np.load('data/backups/train_user_features_autoencoder.csv')

In [7]:
train_data_users = train_data_users.pivot(index='user_id', columns='book_id')
train_data_users.fillna(0, inplace=True)
train_data_users = np.asarray(train_data_users)
train_data_users.shape

(37393, 10000)

In [8]:
full_ratings = pd.read_csv('data/unprocessed/ratings.csv', header=0)
denoised_train_data_users = full_ratings[full_ratings['user_id'].isin(train_users)]

train_data_users['rating'] = train_data_users.apply(lambda x: train_data_users['rating']/5, axis=1)
denoised_train_data_users = denoised_train_data_users.pivot(index='user_id', columns='book_id')
denoised_train_data_users.fillna(0, inplace=True)
denoised_train_data_users = np.asarray(denoised_train_data_users)
denoised_train_data_users.shape

(37393, 10000)

In [9]:
model = DeepCollab(batch_size=128, hidden_layers=5, user_features=True)
model.fit(train_data_users, denoised_train_data_users, train_user_features)

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10000)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10000)        100010000   input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 26)           702         input_2[0][0]                    
_____________________________________

In [10]:
model3 = DeepCollab(batch_size=64, hidden_layers=5, user_features=True)
model3.fit(train_data_users, denoised_train_data_users, train_user_features)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10000)        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 10000)        100010000   input_3[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 26)           702         input_4[0][0]                    
__________________________________________________________________________________________________
concatenat

In [11]:
model4 = DeepCollab(batch_size=32, hidden_layers=5, user_features=True)
model4.fit(train_data_users, denoised_train_data_users, train_user_features)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 10000)        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 10000)        100010000   input_5[0][0]                    
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 26)           702         input_6[0][0]                    
__________________________________________________________________________________________________
concatenat

In [12]:
model5 = DeepCollab(batch_size=32, hidden_layers=5, user_features=True, nodes=2048)
model5.fit(train_data_users, denoised_train_data_users, train_user_features)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 10000)        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 10000)        100010000   input_7[0][0]                    
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 26)           702         input_8[0][0]                    
__________________________________________________________________________________________________
concatenat

In [10]:
train_user_features.shape

(37393, 26)

In [3]:
import numpy as np
from time import time

from keras.layers import Dense, Input, Concatenate
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, TensorBoard

class DeepCollab:
    def __init__(self, batch_size, hidden_layers, user_features, epochs=100, earlystopping=True, nodes=1024):
        
        self.batch_size = batch_size
        self.hidden_layers = hidden_layers
        self.user_features = user_features
        self.epochs = epochs
        self.earlystopping = earlystopping
        self.nodes = nodes

        # Models
        self.autoencoder = None
        

    def fit(self, y, y_noisy, features=None):
        tensorboard = TensorBoard(log_dir="logs/collab/{}".format(time()))
       
        # Input
        input = Input(shape=(y.shape[1],))
        # Encode
        encoded = Dense(y.shape[1], activation='relu')(input)

        if self.user_features:
            user_input = Input(shape=(features.shape[1],))
            user_nodes = Dense(features.shape[1], activation='relu')(user_input)
            merged = Concatenate()([encoded, user_nodes])

            # Hidden Layer
            hidden = Dense(self.nodes, activation='relu')(merged)

        else:
            # Hidden Layer
            hidden = Dense(self.nodes, activation='relu')(encoded)

        if self.hidden_layers == 3:
            hidden = Dense(int(self.nodes/2), activation='relu')(hidden)
            hidden = Dense(self.nodes, activation='relu')(hidden)
        elif self.hidden_layers == 5:
            hidden = Dense(int(self.nodes/2), activation='relu')(hidden)
            hidden = Dense(int(self.nodes/4), activation='relu')(hidden)
            hidden = Dense(int(self.nodes/2), activation='relu')(hidden)
            hidden = Dense(self.nodes, activation='relu')(hidden)

        # Decode
        decoded = Dense(y.shape[1], activation='softmax', bias_initializer='ones')(hidden)

        if self.user_features:
            self.autoencoder = Model([input, user_input], decoded)
            
        else:
            self.autoencoder = Model(input, decoded)

        self.autoencoder.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy'])
        self.autoencoder.summary()

        if self.earlystopping:
            callbacks = [EarlyStopping(), tensorboard]
        else:
            callbacks = [tensorboard]
            
        if self.user_features:

            self.autoencoder.fit([y_noisy, features], y,
                                 epochs=self.epochs,
                                 batch_size=self.batch_size,
                                 validation_split=0.2,
                                 callbacks=callbacks)

        else:

            self.autoencoder.fit(y_noisy, y,
                                 epochs=self.epochs,
                                 batch_size=self.batch_size,
                                 validation_split=0.2,
                                 callbacks=callbacks)

    def predict(self, y_noisy, features):

        if self.user_features:
            predictions = self.autoencoder.predict([y_noisy, features], verbose=1)
        else:
            predictions = self.autoencoder.predict(y_noisy, verbose=1)

        return predictions


# Testing Best Models

In [26]:
test_data_users = pd.read_csv('data/backups/autoencoder_test.csv', header=0)
test_users = list(set(test_data_users['user_id'].tolist()))
test_data_users = test_data_users.pivot(index='user_id', columns='book_id')
test_data_users.fillna(0, inplace=True)
test_data_users = np.asarray(test_data_users)
test_data_users.shape

(16026, 10000)

In [28]:
denoised_test_data_users = full_ratings[full_ratings['user_id'].isin(test_users)]
denoised_test_data_users = denoised_test_data_users.pivot(index='user_id', columns='book_id')
denoised_test_data_users.fillna(0, inplace=True)
denoised_test_data_users = np.asarray(denoised_test_data_users)
denoised_test_data_users.shape

(16026, 10000)

In [27]:
user_features = pd.read_csv('data/user_features_final.csv')
test_user_features = user_features[user_features['user_id'].isin(list(test_users))].copy()
test_user_features['avg_rating'] = test_user_features.apply(lambda x: x['avg_rating']/5, axis=1)
test_user_features.drop('user_id', axis=1, inplace=True)
test_user_features = np.asarray(test_user_features)
test_user_features.shape

(16026, 26)

### 5 Star

In [40]:
predictions = model3.predict(test_data_users, test_user_features)

16026/16026 [==============================] - 3s 215us/step


In [42]:
predictions.shape

(16026, 10000)

In [55]:
predictions[0]

array([4881435. , 5282606.5, 2000445.1, ...,       0. ,       0. ,
             0. ], dtype=float32)

In [ ]:
test_ratings = pd.read_csv('data/backups/autoencoder_test.csv', header=0)
full_ratings = pd.read_csv('data/unprocessed/ratings.csv', header=0)

pred_dfs = []

for index, user in enumerate(test_users):
    train_books = test_ratings[test_ratings['user_id'] == user]['book_id'].tolist()
    all_books = full_ratings[full_ratings['user_id'] == user]['book_id'].tolist()
    test_books = [book for book in all_books if book not in train_books]
    idx = [b + 1 for b in test_books]
    test_book_ratings = full_ratings[full_ratings['book_id'] == user.isin(test_books)]['rating'].tolist()
    raw_predictions = predictions[index]
    raw_predictions = raw_predictions[idx]
    
    raw_filtered_predictions = [rating for ]
    df = pd.DataFrame({'book_id': test_books, 
                       'rating': test_book_ratings,
                       ''
                       
                       'pred_proba': list(raw_predictions[index].values())})
    df['user_id'] = user
    pred_dfs.append(df)
    
predictions = pd.concat(pred_dfs, ignore_index=True)

evaluate_df = test_ratings.merge(predictions, on=['user_id', 'book_id'], how='left')
evaluate_df.head()

# Cold Start

In [47]:
cold_start_features = np.zeros((1, 26))
cold_start_ratings = np.zeros((1, 10000))

In [48]:
cold_start_features.shape

(1, 26)

In [50]:
cold_start = model3.predict(cold_start_ratings, cold_start_features)

1/1 [==============================] - 0s 18ms/step


In [51]:
cold_start

array([[355256.53, 366768.  , 221019.7 , ...,      0.  ,      0.  ,
             0.  ]], dtype=float32)

In [52]:
cold_start.dump('data/backups/cold_start_5star_cdae')